In [1]:
%matplotlib inline

In [2]:
cd '/home/andre/Projects/stocks'

/home/andre/Projects/stocks


In [3]:
from matplotlib import pyplot as plt
import numpy as np
import copy
import torch as th
import torch.nn.functional as F
from torchvision import transforms
from torch.optim import lr_scheduler
import scipy
import sys
import pandas as pd
from Tools.util import progressbar

### Load Vectors Ready for Training

In [4]:
# X = pd.read_pickle('X_train_20180823_all_nobuckets_nopercentiles_nostdn.pickle')
# Y = pd.read_pickle('Y_train_20180823_all_nobuckets_nopercentiles_nostdn.pickle')

In [5]:
import os
import sys
import pandas as pd
import numpy as np
import struct
import datetime
import calendar
import time
import argparse
from Tools.util import progressbar
from Tools import prepareData, torchNN
from Tools import meta5Ibov

In [6]:
nwasted = 120+120+7  # number of lost samples due (shift + EMA's + unknown)
ntraining = 5*8*60 # 8 hours before for training
nscore = 120 # validation samples
nforecast = 120 # forecasting/prediction samples
nneed = ntraining+nscore+nforecast+nwasted # need number of samples
npredict = nscore + nforecast + ntraining # samples for predicting one direction
print(nneed, npredict)

2887 2640


In [7]:
# Working Path for Expert Advisor Metatrader 5
# Use same path that can be read by the expert advisor
datapath = '/home/andre/Projects/stocks/data'
os.chdir(datapath)
# statistical mean and variance from 2013+2018 used to make data
# with variance < 1 and mean close to 0, that is how it works!
stocks_stats = pd.read_csv('stocks_stats_2018.csv', index_col=0)
# selected columns by backtesting best performance accuracy
with open('collumns_selected.txt', 'r') as f: # txt column names divided by spaces
    columns = f.read()
selected_columns = columns.split(' ')[:-1]
#datapath = '/home/andre/.wine/drive_c/Program Files/Rico MetaTrader 5/MQL5/Files'
meta5Ibov.setDataPath(datapath, datapath, verbose=False)
# don't want masterdf to be reused
masterdf = meta5Ibov.loadMeta5Data(suffix='M1.mt5bin',
            cleandays=True, preload=False, verbose=True)
masterdf = meta5Ibov.simpleColumnNames()
percmiss = meta5Ibov.calculateMissing(masterdf)
X, y, Xp = prepareData.GetTrainingPredictionVectors(
    masterdf.copy(), targetsymbol='PETR4_C', verbose=False,
    selected=selected_columns, stats=stocks_stats)

Symbols loaded:
['BBAS3' 'DOL$' 'VALE3' 'ABEV3' 'ITUB4' 'B3SA3' 'BBDC4' 'PETR4' 'WIN@']
percent missing:  0.02973624520526797


In [8]:
X.shape

(504995, 148)

In [9]:
#th.cuda.get_device_name(0)

In [10]:
device = th.device("cuda" if th.cuda.is_available() else "cpu")
input_size = X.shape[1]

In [11]:
# limit = len(X)
# limit = npredict+10 # 10 predictions 
# limit = npredict+100
# nshifts = X.shape[0] - npredict 

In [12]:
len(X)

504995

In [13]:
X.shape

(504995, 148)

In [14]:
len(y)

504995

In [15]:
import importlib
importlib.reload(torchNN)

<module 'Tools.torchNN' from '/home/andre/Projects/stocks/Tools/torchNN.py'>

### Reproducible results with seed

In [16]:
import torch as th
import random
seed=10
np.random.seed(seed)
if str(device) == 'cuda':
    th.backends.cudnn.deterministic = True
random.seed(seed) # pytorch rely on random for weights
th.manual_seed(seed)

In [17]:
#limit = 5*8*60*4*3# 3 months

In [18]:
limit = len(X)

In [19]:
X.shape[0]

504995

In [20]:
%%time
prediction_book = torchNN.backtestPredictions(X[:limit], y[:limit], input_size, device, False)

[################################################################################] 502355/502355
CPU times: user 11h 4min 28s, sys: 9min 35s, total: 11h 14min 4s
Wall time: 11h 16min 29s


In [21]:
len(prediction_book)/limit

0.03712512005069357

In [22]:
len(prediction_book)

18748

In [23]:
prediction_book.to_pickle('PETR4_20180823_all_nobuckets_nopercentiles_no_std_CUDA_NN_EMA120min_shift120_RFy_90_70_percent_pytorch.pickle')

## Need to reinstall CUDA so multiprocessing GPU works

In [22]:
# splitting data to guarantee overlapping multiprocessing
# split data in a batch with limits [start:end]
def paralel_Slide_Predictions_Up_Down(Xtrain, ytrain, start, end):    
    X = Xtrain[start:end]
    y = ytrain[start:end]
    input_size = Xtrain.shape[1]
    #index = sIndex[start:end]        
    prediction_book = Slide_Predictions(X.copy(), y.copy(), input_size)    
    # save recommended orders from starti to endi
    #prediction_book.dropna(inplace=True) # due preallocated size there are nan samples
    return prediction_book
    
def paralel_batch_slices(nprocesses, datasize, nwindow):
    """
    create batch slice indexes for paralel processing
    should write test function for this
    altough it is perfectly working
    """
    batch_size = int(np.floor((datasize-nwindow)/nprocesses))
    nmiss = (datasize-nwindow-batch_size*nprocesses) # samples to complete the non integer division
    # last batch will have more sample to complete the non integer division
    slices = np.array([i*batch_size for i in range(nprocesses)])
    # each batch must have the previous sample size of training + forecast window
    istarts = slices
    iends = slices[1:] + nwindow
    iends = np.append(iends, datasize)
    # first and last batch have different size of samples
    print('batch starts ', istarts)
    print('batch ends ', iends)
    print('batch sizes ', iends-istarts) 
    batches = zip(istarts, iends)
    return batches

###  Using multiprocessing for Pytorch + CUDA
Joblib wont work!  

In [ ]:
import torch.multiprocessing as mp
from multiprocessing import set_start_method

if __name__ == '__main__':
    try:
        set_start_method('spawn')
    except RuntimeError:
        print("is not working")
        pass
    num_processes = 2
    processes = []
    batches = paralel_batch_slices(num_processes, len(X), nwindow)
    for start, end in batches:
        p = mp.Process(target=paralel_Slide_Predictions_Up_Down,
                       args=(X, y, start, end))
        p.start()        
        processes.append(p)
    for p in processes:
        p.join()

batch starts  [     0 249058]
batch ends  [251698 500756]
batch sizes  [251698 251698]
